In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn.linear_model import LinearRegression  
from sklearn.metrics import mean_squared_error
from dmba import stepwise_selection
from dmba import AIC_score


In [57]:
url='C:/Users/andre/Downloads/laboral.csv'
df=pd.read_csv(url,sep=';')

### 1.Emplee el análisis de regresión lineal múltiple que explique la habilidad laboral en términos de las variables res-tantes (actuando como predictoras Xi).

In [58]:
outcome='Y'
modelo=sm.OLS(df[outcome],df[['X1','X2','X3','X4']].assign(const=1))
resultado_1=modelo.fit()
resultado_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     129.7
Date:                Tue, 01 Oct 2024   Prob (F-statistic):           5.26e-14
Time:                        22:29:44   Log-Likelihood:                -67.951
No. Observations:                  25   AIC:                             145.9
Df Residuals:                      20   BIC:                             152.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
X1             0.2957      0.044      6.725      0.000       0.204       0.387
X2             0.0483      0.057      0.853      0.404      -0.070       0.166
X3             1.3060      0.164      7.959      0.000       0.964       1.648
X4             0.5198      0.132      3.940      0.001       0.245       0.795
const       -124.3818      9.941    -12.512      0.000    -145.119    -103.645
==============================================================================
Omnibus:                        3.256   Durbin-Watson:                   1.148
Prob(Omnibus):                  0.196   Jarque-Bera (JB):                1.419
Skew:                           0.139   Prob(JB):                        0.492
Kurtosis:                       1.867   Cond. No.                     2.47e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

x1: por cada aumento de una unidad en x1, la variable Y aumenta 0.2957, esto sugiere una relacion positiva en X1,y, para x2 aumenta un 0.0483 el cual tambien sugiere una relacion positiva resaltando tambien que el coeficiente de x2 es el mas pequeño de todas las X, coefciente x4 es de 0.5198 lo que tambien indica una relacion positiva, aunque mas fuerte que x1, y por ultimo x3 muestra la relacion mas fuerte positiva 1.3060 sugiriendo que un aumento en x4 tiene un impacto mayor en la habilidad laboral.

### 2.Identifique observaciones que puedan considerarse problemáticas (datos atípicos, puntos de balanceo e influ- yentes) y analice si debe eliminarlas de su conjunto de datos o no, justifique. Repita la construcción del modelode regresión si eliminó observaciones.

In [59]:
#Crea un modelo de regresión lineal  usando 'outcome' 
#como variable dependiente y las variables independientes X1, X2, X3 y X4.
modelo=sm.OLS(df[outcome],df[['X1','X2','X3','X4']].assign(const=1))
resultado=modelo.fit()

valores atipicos

In [60]:
from statsmodels.stats.outliers_influence import OLSInfluence

#Crea un objeto de influencia a partir de los resultados del modelo de regresión
influencia=OLSInfluence(resultado)
#Extrae los residuos estandarizados del modelo
residuals=influencia.resid_studentized_internal
# Localiza la fila del DataFrame con el residuo estandarizado más alto y bajo
residuo_max=df.loc[residuals.idxmax(),:]
residuo_min=df.loc[residuals.idxmin(),:]


In [61]:
#Extrae el valor del resultado (Y) correspondiente al residuo mínimo y maximo
outlier_min=residuo_min[outcome]
outlier_maximo=residuo_max[outcome]

df_filtrado = df[(df[outcome] == outlier_min) | (df[outcome] == outlier_maximo)]

print(f'el valor atipico minimo en la variable Y es {outlier_min}\
 el valor atipico maximo en la variable Y es {outlier_maximo} '      )


el valor atipico minimo en la variable Y es 76 el valor atipico maximo en la variable Y es 100 


Los valores atípicos en la variable Y, con un mínimo de 76 y un máximo de 100, indican que hay puntos extremos extraídos de los residuos estandarizados del modelo, que podrían influir en el análisis de los datos. Es importante considerar el contexto de estos valores y su impacto en las estadísticas descriptivas para obtener una comprensión más precisa de la variable Y

los valores atipicos filtrados en el dataframe son valores Y 76 y 100 con indixe 3 y 16 respectivamente

In [62]:
# valores atipicos filtrados en el data set 
df_filtrado

,Y,X1,X2,X3,X4
3,76,101,117,93,95
16,100,104,83,100,102


valores influyentes

In [63]:
#Obtiene las distancias de Cook para cada observación
cooks_d = influencia.cooks_distance[0]
#Calcula el umbral para identificar puntos influyentes
umbral=4/(len(df['Y'])-(df.shape[1]-1))
# Identifica los índices de las observaciones 
# cuya distancia de Cook excede el umbral definido
puntos_de_influencia=np.where(cooks_d>umbral)[0]
#Extrae las distancias de Cook para esos puntos influyentes
cooks_d[puntos_de_influencia]
#Filtra el DataFrame para obtener las observaciones influyentes
observaciones_influyentes = df.iloc[puntos_de_influencia]


el valor influyente tiene un impacto desproporcionado en los resultados de un análisis. Estos valores pueden alterar significativamente las estimaciones de parámetros, los residuos y otros resultados del modelo.

para la observacion 0.191284 es un valor influyente en el modelo ya que esta por encima del valor del umbral 0.19047619, el valor en el dataframe correspondiente seria el indice 15,  con datos 109	109	129	102	108 con variables Y	X1	X2	X3	X4 respectivamente, las posibles acciones aconsiderar es eliminar las observaciones SI SON errores, mantener las observaciones y ver como afectan al modelo final.

In [64]:
# valores influyentes filtrados data set
observaciones_influyentes

,Y,X1,X2,X3,X4
15,109,109,129,102,108


punto de balanceo

In [65]:
umbral_leverage = 2 * (df[['X1', 'X2', 'X3', 'X4']].shape[1] / len(df))
leverage = influencia.hat_matrix_diag

# Identificar los puntos de balanceo
puntos_de_balanceo = np.where(leverage > umbral_leverage)[0]

# Filtrar el DataFrame para obtener las observaciones de balanceo
observaciones_balanceo = df.iloc[puntos_de_balanceo]


Los puntos de balanceo es donde hay un cambio significativo en la dirección de una relación o tendencia, los puntos encontrados se refieren a un cambio en una de las variables independientes (X1, X2, X3 o X4) que afecta a la variable dependiente (Y)

ELIMINAR DATOS ATIPICOS Y INFLUYENTES 

En este estudio, se llevaron a cabo tres modelos de regresión lineal para analizar el conjunto de datos, abordando tanto la inclusión como la exclusión de datos atípicos e influyentes. En la primera fase, se desarrolló un modelo utilizando todos los datos disponibles, lo que permitió observar la tendencia general y la relación entre las variables. Posteriormente, se realizo el modelos con datos atipicos eliminados y por ultimo un modelos borrando los datos influyentes Esta metodología nos permitió comparar la robustez de los tres enfoques y evaluar cómo la presencia de estos datos extremos puede afectar la interpretación de los resultados.

In [66]:
# se crea una copia del data set 
df_eliminar=df.copy()
# se eliminan los valores atipicos encontrados y se revisa de nuevo el modelo 
df_eliminar.drop(index=[3, 16], inplace=True)

In [67]:
modelo_2=sm.OLS(df_eliminar[outcome],df_eliminar[['X1','X2','X3','X4']].assign(const=1))
resultado_2=modelo_2.fit()

In [68]:
# se crea una copia del data set 
df_eliminar_2=df.copy()
# se eliminan los valores atipicos encontrados y se revisa de nuevo el modelo 
df_eliminar_2.drop(index=[15], inplace=True)

In [69]:
modelo_3=sm.OLS(df_eliminar_2[outcome],df_eliminar_2[['X1','X2','X3','X4']].assign(const=1))
resultado_3=modelo_3.fit()


Cuadro resumen de los indicadores de los 3 modelos

In [70]:
cuadro = {
    'resultado1': {'r2': resultado_1.rsquared, 'r2 ajus':resultado_1.rsquared_adj
    ,'f stad':resultado_1.fvalue,'AIC':resultado_1.aic,'Bic': resultado_1.bic},
    'resultado2': {'r2': resultado_2.rsquared, 'r2 ajus':resultado_2.rsquared_adj
    ,'f stad':resultado_2.fvalue,'AIC':resultado_2.aic,
    'Bic': resultado_2.bic,
    }, 'resultado_3': {'r2': resultado_3.rsquared, 'r2 ajus':resultado_3.rsquared_adj
      ,'f stad':resultado_3.fvalue,'AIC':resultado_3.aic,
    'Bic': resultado_3.bic              
    
}}
resumen=pd.DataFrame(cuadro)
resumen

,resultado1,resultado2,resultado_3
r2,0.962892,0.971449,0.966514
r2 ajus,0.955470,0.965105,0.959464
f stad,129.741157,153.114579,137.099986
AIC,145.901138,130.072863,138.187575
Bic,151.995517,135.750334,144.077845


conclucion punto dos 

En el análisis de los tres modelos de regresión lineal, se observa que tanto el R2 como el R2 ajustado presentan variaciones mínimas entre los modelos que incluyen datos atípicos y aquellos que los excluyen.  oscilando entre 0.962892 y 0.971449, y el R2 ajustado también muestra un comportamiento similar, con valores que varían entre 0.955470 y 0.965105.

Estos resultados indican que la inclusión de los datos atípicos e influyentes no afecta de manera significativa la capacidad explicativa de los modelos. Dado que las métricas de ajuste no varían considerablemente, se puede concluir que eliminar estos datos no es necesario para mantener la robustez del modelo. 

### 3. Realice la prueba de significancia del modelo, interprete.

In [71]:
# se saca el p valor del modelo
p_value_modelo = resultado_1.f_pvalue

# Obtener coeficientes y valores p
coeficientes = resultado_1.params
valores_p = resultado_1.pvalues

analisis

Un valor p de 5.26×10−14  esta por debajo del umbral común de 0.05, Esto sugiere que el modelo es significativo en su conjunto y que las variables de X están explicando variabilidad en Y

 por otro lado el R2 con un valor aproximadamente del 96.29% de la variabilidad de la variable dependiente,se puede explicar que el modelo tiene un buen ajuste a los datos

se evaluo las variables x en funcion de los criterios valor p y el coeficiente, siendo el valor p como indicador principal de significancia, en el que se considera el valor p inferior de 0.05  una variable con relevancia.

X1       0.000

X2       0.404

X3       0.000

X4       0.001

el valor p para x1 y x3 es de 0.000, dado que el valor p es menor a 0.05 existe evidencia para concluir que x1 y x3 tiene un efecto significativo en la habilidad laboral, x2  siguen una tendencia de un un valor p de 0.404  respectivamente, indicando que existe un efecto significativo en la habilidad laboral, y para x4 muestra una relacion positiva con un valor p de 0.001.

esto sugiere que las variables mas relevantes para el modelos es x1, x3 y x4.

los coeficientes

X1       0.2957

X2       0.0483

X3       1.3060

X4       0.5198

Todas las variables tienen coeficientes positivos,sin embargo X3 tiene el coeficiente más alto, indicando que es la variable con el efecto más fuerte sobre Y

conclucion 

el análisis revela que X1, X3 y X4 son variables significativas en la habilidad laboral, con X3 mostrando el efecto más fuerte. 

La evidencia sugiere que centrarse en mejorar o entender mejor X1, X3 y X4 podría tener un impacto notable en la habilidad laboral.

### 4. Obtener el coeficiente de determinación y el coeficiente de determinación ajustado. Interprete.

### 6 Realice una selección e variables por el método que prefiera, tome decisiones, explique.

Esta función se encarga de calcular una medida de ajuste del modelo llamada AIC (Akaike Information Criterion), que ayuda a evaluar qué tan bien un modelo se ajusta a los datos en comparación con otros modelos.

la selecccion de las variables sera por medio del criterio AIC, el cual  es útil porque equilibra la mejora en el ajuste del modelo al agregar variables con la necesidad de mantener un modelo simple. Esto asegura que el modelo final sea capaz de generalizar bien a nuevos datos, evitando problemas de sobreajuste y favoreciendo interpretaciones más claras y efectivas.

formula: AIC=2k−2ln(L)

Un AIC más bajo indica un mejor modelo, teniendo en cuenta tanto el ajuste como la complejidad. Un modelo más complejo (más parámetros) puede tener un mejor ajuste, pero el AIC penaliza la complejidad para evitar el sobreajuste.

por lo que un el modelo con el AIC más bajo es considerado el mejor entre los que has evaluado.

In [74]:
from sklearn.linear_model import LinearRegression

# Definir la variable dependiente
y = df[outcome]

# Definir las variables independientes
variables = ['X1', 'X2', 'X3', 'X4']

def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(df[variables], y)  # Ajustar el modelo con las variables X1, X2, X3, X4
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(y, [y.mean()] * len(y), model, df=5)
    return AIC_score(y, model.predict(df[variables]), model)  # Calcular el AIC usando las predicciones del modelo

# Aquí se pasan las variables específicas para la selección
best_model, best_variables = stepwise_selection(variables, train_model, score_model, 
                                                verbose=True)

# Imprimir resultados
print()
print(f'Intercept: {best_model.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(best_variables, best_model.coef_):
    print(f' {name}: {coef:.3f}')


Variables: X1, X2, X3, X4
Start: score=230.25, constant
Step: score=183.42, add X3
Step: score=158.67, add X1
Step: score=146.79, add X4
Step: score=146.79, unchanged None

Intercept: -124.200
Coefficients:
 X3: 1.357
 X1: 0.296
 X4: 0.517
